## LDPC ALgorithm

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [138]:
# generate random n bit and add error
n = 12000 # 12000
qber = 0.02
data = (np.random.rand(n)<0.5)%2
flip = (np.random.rand(n)<qber)%2
data_err = (np.logical_xor(flip,data))%2
#print(np.sum(data_err == data),data_err)
print(sum(flip))

255


### Generate H matrix and convert it to array

In [139]:
dv = 4
dc = 6
m = int(n*dv/dc) # must be integer
H = np.zeros((m,n))
w = int(m/dv) #(must be integer)
H_temp = np.zeros((w,n))
for i in range(w):
    H_temp[i,i*dc:dc*(i+1)] = 1
for i in range(dv):
    H[i*w:w*(i+1),:] = H_temp
    H_temp = np.transpose(np.random.permutation(np.transpose(H_temp)))
H = H.astype("int32")

In [7]:
print(H,H.shape)

[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]] (64, 96)


#### generate indicis matrix

In [140]:
v_ind = np.zeros((dv,n))
cs_ind = np.zeros((dc,m))
for i in range(n):
    v_ind[:,i] = np.reshape(np.nonzero(H[:,i]),-1)
for i in range(m):
    cs_ind[:,i] = np.reshape(np.nonzero(H[i,:]),-1)
v_ind = v_ind.astype("int32")
cs_ind = cs_ind.astype("int32")

In [141]:
k = dv*n
#blf_reg = np.zeros(k)
print(cs_ind)

[[   0    6   12 ...  148 1698  816]
 [   1    7   13 ...  321 1814 2290]
 [   2    8   14 ...  855 6447 2320]
 [   3    9   15 ... 1876 7219 5783]
 [   4   10   16 ... 4043 7393 5804]
 [   5   11   17 ... 7583 8522 7743]]


In [39]:
t1 = np.arange(k)
for i in range(m):
    for j in range(dc):
        w = cs_ind[j,i]
        ind = ind2llr(i,w)
        print(t1[ind])

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]


In [35]:
def ind2llr(i,j):
    '''i<= m row , j<= n colon'''
    temp = np.where(cs_ind[:,i]==j)
    return i*dc+temp[0]

def init_llr(qber,y):
    ''' initialize the LLR, temp has the size of number of bits, and temp one has the size of number of number of
    nonzero element in the generator matrix'''
    temp = np.zeros(k)
    temp1 = np.zeros(n)
    l = np.log((1-qber)/qber)
    p = 1 - 2*y
    for j in range(n):
        for i in v_ind[:,j]:
            ind = ind2llr(i,j)
            temp[ind] = l*p[j]
        temp1[j] = temp[ind]
    return temp, temp1

def phi(x):
   if x<0.000001:
       return 14.509
   if x>14.509:
       return 0.000001
   return np.log((np.exp(x)+1)/(np.exp(x)-1))
 
def cs_update(llr,c):
    temp = np.zeros(k)
    for i in range(m):
        alpha = 1
        l_tot = 0
        #c_b = np.matmul(H,y)%2
        sign = 1-2*c[i]
        for j in cs_ind[:,i]:
            ind = ind2llr(i,j)
            alpha = alpha * np.sign(llr[ind])
            l_tot = l_tot + phi(abs(llr[ind]))
        for j in cs_ind[:,i]:
            ind = ind2llr(i,j)
            alpha1 = alpha * np.sign(llr[ind])
            t1 = l_tot - phi(abs(llr[ind]))
            temp[ind] = alpha1*phi(t1)*sign
    return temp

def v_update(llr,llrb):
    temp = np.zeros(k)
    temp_b = llrb
    for j in range(n):
        for i in v_ind[:,j]:
            ind = ind2llr(i,j)
            temp_b[j] = temp_b[j] + llr[ind]
        for i in v_ind[:,j]:
            ind = ind2llr(i,j)
            temp[ind] = temp_b[j] - llr[ind]
    return temp, temp_b

In [11]:
## temp
#####################
x = data.copy()
y = data_err.copy()
c = np.matmul(H,x)%2
#init
lq = np.zeros(k)
lQ = np.zeros(n)
lq, lQ = init_llr(qber,y)
print(np.sum(x==y))
#######################
####################


94


In [76]:
print(x)
print(y)
print(c)

[1 1 1 1 0 0]
[1 1 1 1 1 0]
[1 1 0 0]


In [79]:
#lq,lQ = v_update(lr,lQ)
#lr = cs_update(lq,c)
print(lq)
print(lQ)
print(lr)

[-0.66248038 -0.66248038 -2.80672173 -0.66248038 -2.80672173  2.80672173
 -2.80672173 -0.66248038  0.66248038 -2.80672173 -2.80672173 -0.66248038]
[-1.73460106 -1.73460106 -3.8788424   0.40964029 -1.73460106  1.73460106]
[-1.07212067 -1.07212067 -1.07212067  1.07212067  1.07212067 -1.07212067
 -1.07212067 -1.07212067  1.07212067  1.07212067  1.07212067  1.07212067]


In [126]:
## temp code for testing (not optmize) 2
x = data
y = data_err
c = np.matmul(H,x)%2
c_b = np.matmul(H,y)%2
d = (c+c_b)%2
#init
blf_reg = np.zeros(k)
bit_blf = np.zeros(n)
blf_reg, bit_reg = init_llr(qber,y)
print(np.sum(x==y))

10863


In [142]:
## temp code for testing (not optmize)
x = data
y = data_err
c = np.matmul(H,x)%2
c_b = np.matmul(H,y)%2
d = (c+c_b)%2
#init
blf_reg = np.zeros(k)
bit_blf = np.zeros(n)
blf_reg, bit_reg = init_llr(qber,y)
print(np.sum(x==y))
def correction(c,y,blf_reg,bit_reg):
    blf_reg1 = cs_update(blf_reg,c)
    blf_reg1, bit_reg1 = v_update(blf_reg1,bit_reg)
    yt = (np.sign(bit_reg1)-1)*(-0.5)
    return yt.astype("int32"),blf_reg1,bit_reg1
y1 = y[:]
       
    

11745


In [143]:
for r in range(30):
    y1,blf_reg,bit_reg1 = correction(c,y1,blf_reg,bit_reg)
    print(np.sum(x==y1)," / 12000")

11948  / 12000
11998  / 12000
12000  / 12000
12000  / 12000
12000  / 12000
12000  / 12000
12000  / 12000
12000  / 12000
12000  / 12000
12000  / 12000
12000  / 12000
12000  / 12000


KeyboardInterrupt: 

In [49]:
np.sign(-2)

-1